In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def sum_above_diagonal(matrix1, matrix2):
    # Check if matrices are square and have the same order
    order1 = len(matrix1)
    order2 = len(matrix2)
    if order1 != len(matrix1[0]) or order2 != len(matrix2[0]) or order1 != order2:
        raise ValueError("Matrices must be square and have the same order.")

    # Initialize sum to 0
    above_diagonal_sum = 0

    # Iterate through each row
    for i in range(order1):
        # Iterate through each column, starting from the diagonal element
        for j in range(i + 1, order1):
            # Sum upper triangular elements
            above_diagonal_sum += matrix1[i][j] + matrix2[i][j]

    return above_diagonal_sum

def above_diagonal_sameness_metric(matrix1, matrix2):
    # Check if matrices are square and have the same order
    order1 = len(matrix1)
    order2 = len(matrix2)
    if order1 != len(matrix1[0]) or order2 != len(matrix2[0]) or order1 != order2:
        raise ValueError("Matrices must be square and have the same order.")

    # Initialize count to 0
    sameness_count = 0

    # Iterate through each row
    for i in range(order1):
        # Iterate through each column, starting from the diagonal element
        for j in range(i + 1, order1):
            # Check if upper triangular elements are the same
            if (matrix1[i][j] + matrix2[i][j]) == 0:
                sameness_count += 1 - 1
            else:
                sameness_count += 1 - 0
    
    return (sameness_count)

# correlation coefficient between the two matrices

def information_linkage_coefficient(matrix1, matrix2):
    term1 = sum_above_diagonal(matrix1, matrix2)
    term2 = above_diagonal_sameness_metric(matrix1, matrix2)
    
    return (term1 - term2)/term2


def above_diagonal_sum_general(matrices):
    # Check if matrices is not empty
    if not matrices:
        raise ValueError("Input list of matrices is empty")

    # Check if all matrices are of the same shape
    shapes = [np.shape(matrix) for matrix in matrices]
    if not all(shape == shapes[0] for shape in shapes):
        raise ValueError("All matrices must have the same shape")
    
    order1 = shapes[0][0]
    
    

    # Initialize the sum matrix with zeros
    above_diagonal_sum = 0

    # Add the above diagonal elements of each matrix
    for matrix in matrices:
        for i in range(order1):
            for j in range(i + 1, order1):
                above_diagonal_sum += matrix[i][j]
                

    return above_diagonal_sum

def sum_matrices(matrices):
    
    if not matrices:
        raise ValueError("List of matrices is empty.")

    result_matrix = np.zeros_like(matrices[0])  # Initialize result matrix with zeros

    for matrix in matrices:
        result_matrix += matrix

    return result_matrix

def above_diagonal_sameness_metric_general(matrices):
    # Check if matrices is not empty
    if not matrices:
        raise ValueError("Input list of matrices is empty")

    # Check if all matrices are of the same shape
    shapes = [np.shape(matrix) for matrix in matrices]
    if not all(shape == shapes[0] for shape in shapes):
        raise ValueError("All matrices must have the same shape")
    
    order1 = shapes[0][0]

    # Initialize count to 0
    sameness_count = 0

    # Iterate through each row
    for i in range(order1):
        # Iterate through each column, starting from the diagonal element
        for j in range(i + 1, order1):
            # Check if above diagonal elements are both zero
            if sum_matrices(matrices)[i][j] == 0:
                sameness_count += 1 - 1
            else:
                sameness_count += 1 - 0
    
    return (sameness_count)

def interlayer_correlation(matrices):
    term1 = above_diagonal_sum_general(matrices)
    term2 = above_diagonal_sameness_metric_general(matrices)
    n_layers = len(matrices)
    
    return (term1 - term2)/((n_layers - 1)*term2)

def embed_time_series(time_series, embedding_dimension, embedding_delay):
    
    n = len(time_series)
    embedded_matrix = np.zeros((n - (embedding_dimension - 1) * embedding_delay, embedding_dimension))

    for i in range(embedding_dimension):
        embedded_matrix[:, i] = time_series[i * embedding_delay : i * embedding_delay + len(embedded_matrix)]

    return embedded_matrix


def euclidean_distance(point1, point2):
 
    return np.linalg.norm(point1 - point2)

def create_closeness_matrix(embedded_time_series, percentage_threshold):
    
    n = embedded_time_series.shape[0]
    closeness_matrix = np.zeros((n, n), dtype=int)

    total_points = n * (n - 1) // 2  # Total number of unique pairs

    # Calculate the threshold distance based on the percentage
    sorted_distances = np.sort([euclidean_distance(embedded_time_series[i], embedded_time_series[j])
                                for i in range(n) for j in range(i + 1, n)])
    threshold_distance_index = int(percentage_threshold * total_points / 100)
    threshold_distance = sorted_distances[threshold_distance_index]

    # Set edges based on the threshold distance
    for i in range(n):
        for j in range(i + 1, n):
            distance = euclidean_distance(embedded_time_series[i], embedded_time_series[j])
            closeness_matrix[i, j] = closeness_matrix[j, i] = 1 if distance < threshold_distance else 0

    return closeness_matrix

def windows(embedded_series, window_length, sliding_step):
    windows = []
    num_windows = int(np.floor((len(embedded_series) - window_length)/sliding_step + 1))
    for i in range(num_windows):
        window = embedded_series[i * (sliding_step) :i * (sliding_step) + window_length]
        windows.append(window)
    return windows

def RecurrencePlot(matrix, name):
    plt.matshow(matrix)
    plt.colorbar()
    plt.title("Adjacency Matrix " + name)
    plt.xlabel("Node (i)")
    plt.ylabel("Node (j)")
    plt.show()

def time_delayed_correlation(dataframe, series1, series2, embedding_dimension, embedding_delay, 
                             window_length, sliding_step, percentage_threshold, correlation_delay):
    
    if len(series1) != len(series2):
        return('time series are of different sizes')
    else:
    
        series1_embedded = embed_time_series(series1, embedding_dimension, embedding_delay) 
        series2_embedded = embed_time_series(series2 , embedding_dimension, embedding_delay) 

        series1_windows = windows(series1_embedded, window_length, sliding_step)
        series2_windows = windows(series2_embedded, window_length, sliding_step)

        corr_df = pd.DataFrame(columns = ['date', 'delayed_correlation'])

        for i in range(abs(correlation_delay), len(series1_windows) - abs(correlation_delay) ):
            m1 = create_closeness_matrix(series1_windows[i], percentage_threshold)
            m2 = create_closeness_matrix(series2_windows[i + correlation_delay], percentage_threshold)
    
            window_index1 = dataframe.index[series1 == series1_windows[i][0][0]].tolist()
            date = dataframe['Date'][window_index1[0]]
            correlation1 = interlayer_correlation([m1, m2])
            print(date, correlation1)
    
            corr_df.loc[len(corr_df)] = [date, correlation1]
        
        return corr_df
    


In [ ]:
df = pd.read_csv('stock data for RNs.csv')

In [ ]:
df['AXISBANK.NS'] = df['AXISBANK.NS'].interpolate()
testing_df = df[['Date','HDFCBANK.NS', 'AXISBANK.NS', 'year']]
new_df = testing_df.copy()
for i in range(1, len(testing_df)):
    new_df['HDFCBANK.NS'][i] = math.log(testing_df['HDFCBANK.NS'][i]/testing_df['HDFCBANK.NS'][i-1])
    new_df['AXISBANK.NS'][i] = math.log(testing_df['AXISBANK.NS'][i]/testing_df['AXISBANK.NS'][i-1])
    
new_df = new_df.iloc[1:]
new_df = new_df[new_df['year'] >= 2017]

hdfc_embedded = embed_time_series(new_df['HDFCBANK.NS'], 8, 5) 
axis_embedded = embed_time_series(new_df['AXISBANK.NS'] , 8, 5) 

hdfc_windows = windows(hdfc_embedded, 250, 5)
axis_windows = windows(axis_embedded, 250, 5)

correlation_df = pd.DataFrame(columns = ['date', 'correlation'])

for i in range(len(hdfc_windows)):
    m1 = create_closeness_matrix(hdfc_windows[i], 5)
    m2 = create_closeness_matrix(axis_windows[i], 5)
    
    window_index = new_df.index[new_df['HDFCBANK.NS'] == hdfc_windows[i][0][0]].tolist()
    date = new_df['Date'][window_index[0]]
    correlation = interlayer_correlation([m1, m2])
    print(date, correlation)
    
    correlation_df.loc[len(correlation_df)] = [date, correlation]
    

for i in range(len(correlation_df)):
    correlation_df['date'][i] = correlation_df['date'][i].split()[0].split('-')[0] + '-' + correlation_df['date'][i].split()[0].split('-')[1]
    
correlation_df['date'] = pd.to_datetime(correlation_df['date'])

# Plotting the elements of the list
plt.figure(figsize = (10, 10))
plt.plot(correlation_df['date'], correlation_df['correlation'] , marker = 'o', linestyle = '-')
plt.title('correlation over time (HDFC AND AXIS BANK)')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
#plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.xlabel('time period')
plt.ylabel('correlation')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(correlation_df['date'], correlation_df['correlation'] , marker = 'o', linestyle = '-')
plt.title('correlation over time (HDFC AND AXIS BANK)')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
#plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.xlabel('time period')
plt.ylabel('correlation')
plt.show()